# Consolidate AMULET outputs into a single doublet list
This script concatenates all the amulet outputs to a single file

In [1]:
import os
import glob
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


# Load AMULET doublet calls

In [4]:
# Make sure this matches the number of samples you expect
base_dir = "/path/to/output/"
amulet_bc_paths = glob.glob(os.path.join(base_dir, "*", "atac/amulet/doublet_barcodes.txt"))
amulet_bc_paths[:5], len(amulet_bc_paths)

(['/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/3_sample_qc/dm31a/atac/amulet/doublet_barcodes.txt',
  '/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/3_sample_qc/dm42b/atac/amulet/doublet_barcodes.txt',
  '/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/3_sample_qc/25-1/atac/amulet/doublet_barcodes.txt',
  '/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/3_sample_qc/35-1/atac/amulet/doublet_barcodes.txt',
  '/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/3_sample_qc/14-1/atac/amulet/doublet_barcodes.txt'],
 56)

In [5]:
# Concat
amulet_bcs = []
num_doublets = {}
for path in amulet_bc_paths:
    sample = path.split("/")[-4]
    curr_bcs = pd.read_csv(path, sep="\t", header=None)[0].tolist()
    curr_bcs = [sample + "#" + bc for bc in curr_bcs]
    num_doublets[sample] = len(curr_bcs)
    amulet_bcs.extend(curr_bcs)
amulet_bcs[:5], len(amulet_bcs)

(['dm31a#AAAGCAAGTGTGTGGT-1',
  'dm31a#AAAGCACCAGGATGGC-1',
  'dm31a#AAAGCGGGTCATGCAA-1',
  'dm31a#AAAGGTTAGGCAAGTA-1',
  'dm31a#AACCTAATCAGCTAAC-1'],
 29106)

# Write

In [6]:
# Print total number of samples with doublets
len(num_doublets)

56

In [7]:
# Average per sample
avg_doublets = sum(num_doublets.values()) / len(num_doublets)
avg_doublets

519.75

In [9]:
with open(os.path.join(base_dir, "amulet_bcs.txt"), "w") as f:
    f.write("\n".join(amulet_bcs))

In [12]:
# Save number of doublets per sample
num_doublets_per_sample = pd.Series(num_doublets)
num_doublets_per_sample.name = "num_doublets"
num_doublets_per_sample.index.name = "sample"
num_doublets_per_sample.to_csv(os.path.join(base_dir, "num_doublets_per_sample.csv"))

# DONE!

---